In [1]:
import sys
sys.path.append("./")

In [2]:
from utils.model_setup import *
from utils.dataset_setup import *
from utils.experiment_setup import *
from utils.save_results import *

In [3]:
path_to = "./data/"
path_to_experiments = "./experiments/"

## Excuting experiments

In [4]:
def save_state(batch_saved=0):
  state = {
    'Model': model_code,
    'Database': dataset_code,
    'Experimento': experiment_code,
    'Execucao': execution,
    'MaxExecucao': execution_limit,
    'Batch': batch_saved
  }
  with open(path_to + 'EstadoExperimentosChat.csv', mode='w') as file:
    print("Saving state: " + str(state))
    writer = csv.DictWriter(file, fieldnames=state.keys())
    writer.writeheader()
    writer.writerow(state)

In [5]:
def run_model(essays_dataset, exp, ini, end, model_name):
  essays_outputs = []

  for i in range(ini, end):
    if i == 34 and dataset_name == ("essaysGrade1000"):
      print("Skipping essay number: " + str(i))
      continue

    if (i == 101 or i == 131 or i == 143) and dataset_name == ("extended2024"):
      print("Skipping essay number: " + str(i))
      continue

    if (i == 97 or i == 98 or i == 112) and dataset_name == ("propor2024"):
      print("Skipping essay number: " + str(i))
      continue

    essay_data = essays_dataset[i]
    essay = essay_data["essay_text"]
    prompt = essay_data["id_prompt"].replace("-", " ")
    print("It will evaluated essay with number:" + str(i))
    prompt = exp.build_prompt(essay, prompt)
    grades, response = made_question(model_name, chat, prompt)

    print("Evaluated essay with number: " + str(i))

    model_key = model_name.split("-")[0]
    grades_key = model_key + "_grades"
    output_key = model_key + "_output"

    output = {
      "id": essay_data["id"],
      "essay_text": essay_data["essay_text"],
      "id_prompt": essay_data["id_prompt"],
      "grades": essay_data["grades"],
      grades_key: grades,
      output_key: response,
    }

    essays_outputs.append(output)

  return essays_outputs

In [6]:
def run_experiment(exp, essays_dataset, start, limit, batch, path_to_save, filename_to_save, model_name):
  end = start
  for ini in range(start, limit-batch, batch):
    end = ini + batch

    
    try:
      essays_outputs = run_model(essays_dataset, exp, ini, end, model_name)
    except Exception as e:
      print(f"There was an exception on run_experiment!!: {e}")
      time.sleep(180)
      chat = model_setup(model_name)
      essays_outputs = run_model(essays_dataset, exp, ini, end, run_model)
    

    if ini == 0:
      create_csv_header(path_to_save, filename_to_save, essays_outputs[0].keys())

    save_data(path_to_save, filename_to_save, essays_outputs)

    print("Processed essays from " + str(ini) + " to: " + str(end))
    save_state(end)

  print("End value is: ", end)
  print("Limit value is:", limit)

  if limit !=  end:
    essays_outputs = run_model(essays_dataset, exp, end, limit, model_name)
    save_data(path_to_save, filename_to_save, essays_outputs)
    print("Processed essays from " + str(end) + " to: " + str(limit))

## Setup

In [7]:
use_experiment_state = True
model_code = -1
dataset_code = -1
experiment_code = -1
execution = 1
batch_saved = 0
execution_limit = 5

if use_experiment_state:

  with open(path_to + 'EstadoExperimentosChat.csv', mode='r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        state = row  # Each row is a dictionary

  print(state)

  model_code = int(state['Model'])
  dataset_code = int(state['Database'])
  experiment_code = int(state['Experimento'])
  execution = int(state['Execucao'])
  execution_limit = int(state['MaxExecucao'])
  batch_saved = int(state['Batch'])


{'Model': '2', 'Database': '2', 'Experimento': '1', 'Execucao': '4', 'MaxExecucao': '5', 'Batch': '100'}


In [8]:
"""
Setup Model
"""
model_name, model_code = choose_model(model_code)
chat = model_setup(model_name)

Choosen Model: llama-3.2-90b-text-preview


In [9]:
"""
Setup Dataset
"""
dataset_name, dataset_code = setup_dataset(dataset_code)
essays_dataset = getDataset(path_to, dataset_name)

Dataset escolhido: aes_enem_dataset


In [10]:
while experiment_code <= 5:
  """
  Setup Experiment
  """
  experiment_name, experiment_code = choose_experiment(experiment_code)
  exp = setup_experiment(path_to_experiments, experiment_name)

  save_state(batch_saved)

  while execution <= execution_limit:
    print()
    print()
    print("Execution number: " + str(execution))
    print()
    print()

    start = batch_saved
    path_to_save = create_experiment_folder(path_to, model_name, experiment_name, dataset_name)
    filename_to_save = build_filename_to_save(model_name, experiment_name, dataset_name, str(execution))

    batch = 10
    limit = len(essays_dataset)

    run_experiment(exp, essays_dataset, start, limit, batch, path_to_save, filename_to_save, model_name)
    batch_saved = 0 # IMPORTANT WHEN MODEL STOP IN THE MIDDLE
    execution += 1
    save_state()

  experiment_code += 1
  execution = 0
  save_state()


Experimento escolhido: exp1
Create exp1
Prompt template: 
Corrija a seguinte redação, seguindo o método de avaliação do ENEM, entregue apenas a nota quantitativa entre 0 e 200 para cada competência e a nota final no seguinte formato: [x, y, z, w, t, x+y+z+w+t]. Não quero texto na resposta, apenas as notas numéricas para cada competência e o total 

Tema:<TEMA> 
Redação: <REDAÇÃO>
Saving state: {'Model': 2, 'Database': 2, 'Experimento': 1, 'Execucao': 4, 'MaxExecucao': 5, 'Batch': 100}


Execution number: 4


Folder './data/results/llama-3.2-90b-text-preview' already exists.
Folder './data/results/llama-3.2-90b-text-preview/exp1' already exists.
Folder './data/results/llama-3.2-90b-text-preview/exp1/aes_enem_dataset' already exists.
CSV with the answer will be saved in: aes_enem_dataset-llama-3.2-90b-text-preview-exp1-4
It will evaluated essay with number:100
Testing if it is arriving here!!
Evaluated essay with number: 100
It will evaluated essay with number:101
Testing if it is arrivi

KeyError: <function run_model at 0x7f6c3d834af0>

In [ ]:
"""
Setup Experiment
"""
experiment_name, experiment_code = choose_experiment(experiment_code)
exp = setup_experiment(path_to_experiments, experiment_name)

In [ ]:
essay_data = essays_dataset[2]
essay = essay_data["essay_text"]
prompt = essay_data["id_prompt"].replace("-", " ")
prompt = exp.build_prompt(essay, prompt)

model_prompt = exp.build_prompt(essay, prompt)

In [ ]:
grades, response = made_question(model_name, chat, model_prompt)

In [ ]:
print